### **Curating visium_slide3.h5ad**

Article:   A human embryonic limb cell atlas resolved in space and time

DOI: https://doi.org/10.1038/s41586-023-06806-x

Data Source : https://developmental.cellatlas.io/embryonic-limb

##### **Mount farm**

mount-farm

##### **Packages required for curation**

In [ ]:
#Import all packages required for curation

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import csr_matrix
import anndata as ad
import os
import subprocess
import math

### **Curation Schema**

##### **X (Matrix Layers)**

##### **AnnData object**

In [ ]:
# Load the AnnData object

In [ ]:
adata = sc.read_h5ad('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/limb_cell_atlas/Data/New_data/visium_slide3.h5ad')

In [ ]:
# View the AnnData object

In [ ]:
adata

##### **X- expression matrix**

In [ ]:
# View the expression matrix of the anndata object

In [ ]:
adata.X

In [ ]:
# Print the matrix to check whether they are normalized counts or raw counts. if the matrix has floating numbers,they are normalized counts.if they are integers, they are raw counts.

In [ ]:
print(adata.X)

##### **Variables(var)**

In [ ]:
#View the var of anndata and raw object

In [ ]:
adata.var

In [ ]:
#Store ensembl ids in a new column in adata.var by matching gene symbols and ensembl ids from the gene information file

In [ ]:
adata.var['gene_symbols'] = adata.var_names

In [ ]:
adata.var_names = adata.var['ensg_ids']

In [ ]:
adata.var

In [ ]:
# load the approved genes file

In [ ]:
approved_genes = pd.read_csv('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/Endometrium_reference_integrated_atlas/genes_approved.csv')

In [ ]:
# Create a dictionary from the approved genes file using the symbols and feature id columns.

In [ ]:
genedict = {key: 1 for key in list(approved_genes.feature_id)}

In [ ]:
genedict

In [ ]:
len(genedict)

In [ ]:
# Filter out the genes which are not in the approved genes file

In [ ]:
var_to_keep_adata = [x for x in adata.var_names if (x in genedict)]

In [ ]:
len(var_to_keep_adata)

In [ ]:
# Modify the anndata object by filtering out the filtered genes. copy the index column values to a new column called gene_symbols

In [ ]:
adata = adata[:, var_to_keep_adata].copy()

In [ ]:
#  View the var

In [ ]:
adata.var

feature is filtered

In [ ]:
# view var

In [ ]:
adata.var['feature_is_filtered'] = [False] * len(adata.var)

In [ ]:
adata.var

In [ ]:
del adata.var['ensg_ids']
del adata.var['gene_symbols']

#### **Observations(obs) (Cell metadata)**

In [ ]:
#view obs

In [ ]:
adata.obs

In [ ]:
adata.obs.columns

#### **assay_ontology_term_id**

In [ ]:
# add the assay_ontology_term_id column

In [ ]:
adata.obs['assay_ontology_term_id'] = ['EFO:0010961'] * len(adata.obs)

In [ ]:
# change datatype of the column

In [ ]:
adata.obs['assay_ontology_term_id'] = adata.obs['assay_ontology_term_id'].astype('category')

In [ ]:
# view adata.obs

In [ ]:
adata.obs

#### **cell_type_ontology_term_id**

In [ ]:
#get the column in adata.obs related. to cell type annotation

In [ ]:
adata.obs.columns

In [ ]:
adata.obsm

In [ ]:
c2l_columns = [col for col in adata.obs.columns if col.startswith('C2L')]

In [ ]:
c2l_columns

In [ ]:
adata.obs['max_c2l_column'] = adata.obs[c2l_columns].idxmax(axis=1)

In [ ]:
adata.obs['max_c2l_column_value'] = adata.obs[c2l_columns].max(axis=1)

In [ ]:
adata.obs

In [ ]:
list(adata.obs['max_c2l_column'].unique())

In [ ]:
import pandas as pd

# Assuming adata.obs['col'] is a pandas Series object
adata.obs['max_c2l_column'] = adata.obs['max_c2l_column'].fillna('NaN').astype(str)


In [ ]:
list(adata.obs['max_c2l_column'].unique())

In [ ]:
mapping= {'C2L: Teno' :'CL:0000388',
 'C2L: SMC':'CL:0000192',
 'C2L: F10+DermFibroProg':'CL:0002551',
 'C2L: MYL3+MyoC':'CL:0000187',
 'C2L: HOXC5+DermFibroProg':'CL:0002551',
 'C2L: MFAP5+Fibro':'CL:0000057',
 'C2L: Pericyte':'CL:0000669',
 'C2L: HyperChon':'CL:0000743',
 'C2L: MYH3+MyoC':'CL:0000187',
 'C2L: Periderm':'CL:0000078',
 'C2L: Perimysium':'CL:0002320',
 'C2L: DermFibro':'CL:0002551',
 'C2L: Schwann':'CL:0002573',
 'C2L: ADH+Fibro':'CL:1001609',
 'C2L: RestingChon':'CL:0000138',
 'C2L: MesCond':'CL:0000138',
 'C2L: DefErythro':'CL:0000232',
 'C2L: ArtiChon':'CL:1001607',
 'C2L: Basal':'CL:0000646',
 'C2L: PAX7+MyoProg':'CL:0000187',
 'C2L: Macro':'CL:0000235',
 'C2L: ArterialEndo':'CL:1000413',
 'C2L: DefReticulo':'CL:0000558',
 'C2L: VenousEndo':'CL:0002543',
 'C2L: OsteoB':'CL:0000062',
 'C2L: NeuralFibro':'CL:0000057',
 'C2L: PrimErythro1':'CL:0002355',
 'C2L: AER-Basal':'CL:0000646',
 'C2L: ChondroProg':'CL:0000138',
 'C2L: SMProg':'CL:0000192',
 'NaN':'unknown',
 'C2L: DistalMes':'CL:0008019',
 'C2L: Neuronal':'CL:0000540',
 'C2L: PrehyperChon':'CL:0000138', 
 'C2L: Megakaryo' :'CL:0000556',
 'C2L: LymphEndo' : 'CL:0002138',
 'C2L: Mes4':'CL:0008019',
 'C2L: STMN2+Fibro':'CL:0002551',
 'C2L: InterZone':'CL:0008019',
 'C2L: SynapSchwann':'CL:0002573',
 'C2L: DC2':'CL:0000990',
 'C2L: SchwannProg':'CL:0002573',
 'C2L: PAX3+MyoProg':'CL:0000515',
 'C2L: Mes3':'CL:0008019',
 'C2L: Perichon':'CL:0000058',
 'C2L: TenoProg':'CL:0000388',
 'C2L: ProlifChon':'CL:0000138',
 'C2L: RDH10+DistalMes':'CL:0008019',
 'C2L: PrimErythro2':'CL:0002355',
 'C2L: TransMes':'CL:0008019',
 'C2L: Mes2':'CL:0008019'}


In [ ]:
# create a dictionary of cell type and ontology term

In [ ]:
# add the cell_type_ontology_term_id column

In [ ]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['max_c2l_column'].map(mapping)

In [ ]:
# change datatype of the column

In [ ]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].astype('category')

In [ ]:
list(adata.obs['cell_type_ontology_term_id'].unique())

In [ ]:
adata.obs

#### **donor_id**

In [ ]:
#identify the column in adata.obs which provides donor information

In [ ]:
adata.obs.columns

In [ ]:
list(adata.obs['library_id'].unique())

In [ ]:
# add the donor_id column

In [ ]:
adata.obs['donor_id'] = adata.obs['library_id']

In [ ]:
# change datatype of the column

In [ ]:
adata.obs['donor_id'] = adata.obs['donor_id'].astype('category')

In [ ]:
# view unique values of donor_id column

In [ ]:
list(adata.obs['donor_id'].unique())

In [ ]:
#view obs

In [ ]:
adata.obs

In [ ]:
adata.obs.columns

In [ ]:
adata.obs

#### **development_stage_ontology_term_id**

In [ ]:
# identify the column in adata which corresponds to age

In [ ]:
# add the development_stage_ontology_term_id column

In [ ]:
mapping = {'WSSS_THYst9383359':'PCW8.1',
'WSSS_THYst9383360':'PCW8.1',
'WSSS_THYst9383361':'PCW8.1',
'WSSS_THYst9383362':'PCW8.1',
'WSSS_THYst9699523':'PCW7.0',
'WSSS_THYst9699524':'PCW7.0',
'WSSS_THYst9699525':'PCW6.2',
'WSSS_THYst9699526':'PCW5.6'}

In [ ]:
adata.obs['stage'] = adata.obs['library_id'].map(mapping)

In [ ]:
mapping= {'PCW8.1' :'HsapDv:0000030','PCW7.0':'HsapDv:0000026','PCW6.2':'HsapDv:0000024','PCW5.6':'HsapDv:0000023'}

In [ ]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['stage'].map(mapping)

In [ ]:
# change datatype of the column

In [ ]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['development_stage_ontology_term_id'].astype('category')

In [ ]:
# view unique values of development_stage_ontology_term_id column

In [ ]:
list(adata.obs['development_stage_ontology_term_id'].unique())

In [ ]:
# view adata.obs

In [ ]:
adata.obs

#### **disease_ontology_term_id**

In [ ]:
# Assign normal since all are healthy patients

In [ ]:
# add the disease_ontology_term_id column

In [ ]:
adata.obs['disease_ontology_term_id'] = ['PATO:0000461'] * len(adata.obs)

In [ ]:
#change data type of column

In [ ]:
adata.obs['disease_ontology_term_id'] = adata.obs['disease_ontology_term_id'].astype('category')

In [ ]:
# view obs

In [ ]:
adata.obs

#### **is_primary_data**

In [ ]:
adata.obs['is_primary_data'] = [True] * len(adata.obs)

In [ ]:
adata.obs

In [ ]:
#change data type of column

In [ ]:
adata.obs['is_primary_data'] = adata.obs['is_primary_data'].astype('bool')

#### **organism_ontology_term_id**

In [ ]:
# assign organism id 

In [ ]:
adata.obs['organism_ontology_term_id'] = ['NCBITaxon:9606'] * len(adata.obs)

In [ ]:
#change data type of column

In [ ]:
adata.obs['organism_ontology_term_id'] = adata.obs['organism_ontology_term_id'].astype('category')

In [ ]:
# view obs

In [ ]:
adata.obs

#### **self_reported_ethnicity_ontology_term_id**

In [ ]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = ['unknown'] * len(adata.obs)

In [ ]:
# change data type

In [ ]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = adata.obs['self_reported_ethnicity_ontology_term_id'].astype('category')

In [ ]:
# view obs

In [ ]:
adata.obs

In [ ]:
list(adata.obs['self_reported_ethnicity_ontology_term_id'].unique())

#### **sex_ontology_term_id**

In [ ]:
mapping = {'5386STDY7537944':'F',
           '5478STDY7717491':'M',
'5478STDY7717492':'M',
'5478STDY7652318':'M',
'5386STDY7557336':'M',
'5386STDY7557337':'M',
'5386STDY7557335':'F',
'FCAImmP7536758':'F',
'FCAImmP7536759':'F',
'5478STDY7980348':'M',
'5478STDY7980349':'M',
'5478STDY7935101':'F',
'5478STDY7935102':'M',
'WSSS_THYst9384953':'M',
'WSSS_THYst9384954':'M',
'WSSS_THYst9384955':'M',
'WSSS_THYst9384956':'M',
'WSSS_THYst9384957':'M',
'WSSS_THYst9384958':'M',
'WSSS_THYst8796437':'M',
'WSSS_THYst8796438':'M',
'WSSS_THYst8796439':'M',
'WSSS_THYst8796440':'M',
'WSSS_THYst8796441':'M',
'WSSS_THYst8796442':'M',
'WSSS_THYst9383359':'M',
'WSSS_THYst9383360':'M',
'WSSS_THYst9383361':'M',
'WSSS_THYst9383362':'M',
'WSSS_THYst9699523':'F',
'WSSS_THYst9699524':'F',
'WSSS_THYst9699525':'F',
'WSSS_THYst9699526':'unknown'}

In [ ]:
adata.obs['sex'] = adata.obs['library_id'].map(mapping)

In [ ]:
mapping= {'F': 'PATO:0000383', 'M': 'PATO:0000384', 'unknown':'unknown'}

In [ ]:
adata.obs['sex_ontology_term_id'] = adata.obs['sex'].map(mapping)

In [ ]:
adata.obs['sex_ontology_term_id'] = ['PATO:0000384'] * len(adata.obs)

In [ ]:
# change data type

In [ ]:
adata.obs['sex_ontology_term_id'] = adata.obs['sex_ontology_term_id'].astype('category')

In [ ]:
adata.obs

#### **suspension_type**

In [ ]:
# since visium suspension type is 'na'

In [ ]:
adata.obs['suspension_type'] = ['na'] * len(adata.obs)

In [ ]:
# change data type

In [ ]:
adata.obs['suspension_type'] = adata.obs['suspension_type'].astype('category')

In [ ]:
# view obs

In [ ]:
adata.obs

#### **tissue_type**

In [ ]:
adata.obs['tissue_type'] = ['tissue'] * len(adata.obs)

In [ ]:
adata.obs['tissue_type'] = adata.obs['tissue_type'].astype('category')

#### **tissue_ontology_term_id**

In [ ]:
list(adata.obs['library_id'].unique())

In [ ]:
mapping= {'WSSS_THYst9383360':'UBERON:0000978', 'WSSS_THYst9383361':'UBERON:0000978', 'WSSS_THYst9383362':'UBERON:0000978'}

In [ ]:
adata.obs['tissue_ontology_term_id'] = adata.obs['library_id'].map(mapping)

In [ ]:
adata.obs['tissue_ontology_term_id'] = adata.obs['tissue_ontology_term_id'].astype('category')

In [ ]:
list(adata.obs['tissue_ontology_term_id'].unique())

In [ ]:
# view obs

In [ ]:
adata.obs

In [ ]:
adata.obs.columns

In [ ]:
del adata.obs['barcode']
del adata.obs['max_c2l_column']
del adata.obs['max_c2l_column_value']
del adata.obs['array_row']
del adata.obs['array_col']
del adata.obs['library_id']
del adata.obs['sex']

#### **obsm (Embeddings)**

In [ ]:
adata.obsm

In [ ]:
adata.obsm.keys()

#### **uns (Dataset Metadata)**

In [ ]:
adata.uns

In [ ]:
adata.uns['image_caption'] = 'Shown here is an image of ten micron thick cryosections of whole embryonic limb samples stained with H&E'

In [ ]:
adata.uns['title'] = 'visium_slide3'

In [ ]:
adata.uns['default_embedding'] = 'X_spatial'

In [ ]:
adata.uns.keys()

### **Final checks and adjustments**

In [ ]:
adata

In [ ]:
adata.obs.dtypes

In [ ]:
dty = pd.DataFrame(adata.var.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.var[c] = adata.var[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.var[c] = adata.var[c].astype('int32') 
    print(f"changed {c} from int64 to int32")

In [ ]:
dty = pd.DataFrame(adata.obs.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.obs[c] = adata.obs[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.obs[c] = adata.obs[c].astype('int32') 
    print(f"changed {c} from int64 to int32")
for c in dty[dty['dtype'] == 'object'].index.values:
    adata.obs[c] = adata.obs[c].astype('category') 
    print(f"changed {c} from object to category")

In [ ]:
adata.obs

In [ ]:
adata.obs.columns

In [ ]:
adata.var

In [ ]:
adata.obs

In [ ]:
adata.obs.columns

In [ ]:
#check the format of expression matrix

In [ ]:
adata.X

In [ ]:
#write the curated object to final_objects folder

In [ ]:
adata.write('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/limb_cell_atlas/Final_objects/visium_slide3.h5ad', compression = 'gzip')